In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install datasets transformers

     |████████████████████████████████| 225kB 34.3MB/s 
     |████████████████████████████████| 2.3MB 44.6MB/s 
     |████████████████████████████████| 245kB 44.2MB/s 
     |████████████████████████████████| 112kB 56.1MB/s 
     |████████████████████████████████| 901kB 42.6MB/s 
     |████████████████████████████████| 3.3MB 48.7MB/s 


In [ ]:
from datasets import load_dataset
datasets = load_dataset("csv", data_files={"train":'/content/drive/MyDrive/Train and valid spli/train.csv',"test":'/content/drive/MyDrive/Train and valid spli/test.csv',"valid":'/content/drive/MyDrive/Train and valid spli/val.csv'},encoding='utf-8')

Using custom data configuration default-a86a164abac00962


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a86a164abac00962/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'labels', 'tweets'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'labels', 'tweets'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'labels', 'tweets'],
        num_rows: 36370
    })
})

In [ ]:
datasets=datasets.remove_columns('Unnamed: 0')

In [ ]:
model_checkpoint = "xlm-roberta-base"

In [ ]:
from transformers import XLMRobertaForCausalLM
model=XLMRobertaForCausalLM.from_pretrained(model_checkpoint)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.1MB 29.4MB/s 


In [ ]:
from transformers import XLMRobertaTokenizerFast
    
tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tweets"],padding=True,truncation=True,max_length=128)

In [ ]:
datasets=datasets.remove_columns('Unnamed: 0.1')

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'tweets'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['labels', 'tweets'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['labels', 'tweets'],
        num_rows: 36370
    })
})

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=3)

In [ ]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'tweets'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'tweets'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'tweets'],
        num_rows: 36370
    })
})

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns('labels')

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns('tweets')

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=3,
)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
import torch
device = torch.device('cuda:0')
model =model.to(device)

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["valid"]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.069700,0.070717
2,0.062000,0.062498
3,0.059500,0.060020


TrainOutput(global_step=24471, training_loss=0.09340754646892703, metrics={'train_runtime': 20985.8645, 'train_samples_per_second': 1.166, 'total_flos': 0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 176128, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 411860992, 'train_mem_gpu_alloc_delta': 3373098496, 'train_mem_cpu_peaked_delta': 371924992, 'train_mem_gpu_peaked_delta': 5989954560})

In [ ]:
import math
eval_results = trainer.evaluate(lm_datasets['test'])
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.06


In [ ]:
import os
output_dir ="/content/drive/MyDrive/CasualLM"
output_model_file = os.path.join(output_dir, 'pytorch_model.bin')
output_config_file = os.path.join(output_dir, 'config.json')
model_to_save = model.module if hasattr(model, 'module') else model
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

('/content/drive/MyDrive/CasualLM/sentencepiece.bpe.model',)

In [ ]:
output_dir ="/content/drive/MyDrive/CasualLM"

In [ ]:
from transformers import XLMRobertaForSequenceClassification

In [ ]:
loaded_model=XLMRobertaForSequenceClassification.from_pretrained(output_dir,num_labels=2)

Some weights of the model checkpoint at /content/drive/MyDrive/CasualLM were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/CasualLM and are newly initialized: ['classifier.dense.weigh

In [ ]:
from transformers import XLMRobertaTokenizerFast

In [ ]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained(output_dir) 

In [ ]:
dataset = load_dataset("csv", data_files={"train":'/content/drive/MyDrive/split_dataset/train.csv', "validation":'/content/drive/MyDrive/split_dataset/valid.csv',"test":'/content/drive/MyDrive/split_dataset/test.csv'})

Using custom data configuration default-3221b56a9807690e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3221b56a9807690e/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
dataset=dataset.remove_columns('Unnamed: 0')

In [ ]:
dataset['train'][0]

{'labels': 0,
 'tweets': 'ather farouqui general secretary of ghar emphasis on preservation of of in his'}

In [ ]:
def preprocess_function(examples):
      return tokenizer(examples['tweets'], truncation=True,padding=True,max_length=128)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset=encoded_dataset.remove_columns('tweets')

In [ ]:
import torch
device = torch.device('cuda:0')
loaded_model = loaded_model.to(device)
args = TrainingArguments(
    "FInetuned3",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 115000
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 21370
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 30000
    })
})

In [ ]:
args.place_model_on_device

True

In [ ]:
from sklearn.metrics import accuracy_score,label_ranking_average_precision_score,hamming_loss,f1_score,roc_auc_score

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    y_true=labels
    y_pred=predictions
    return accuracy_score(labels,predictions),f1_score(y_true, y_pred, average='micro'),roc_auc_score(y_true, y_pred, average='micro'),label_ranking_average_precision_score(y_true, y_pred),hamming_loss(y_true, y_pred)

In [ ]:
import numpy as np

In [ ]:
trainer = Trainer(
    loaded_model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,   
)

In [ ]:
import numpy as np

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.322200,0.327783,0.870426,0.860735
2,0.298700,0.300265,0.888723,0.884887
3,0.251300,0.293239,0.890220,0.886754


TrainOutput(global_step=21564, training_loss=0.3184872638579977, metrics={'train_runtime': 9123.5158, 'train_samples_per_second': 2.364, 'total_flos': 0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 725004288, 'train_mem_gpu_alloc_delta': 3350130176, 'train_mem_cpu_peaked_delta': 119070720, 'train_mem_gpu_peaked_delta': 1523655680})

In [ ]:
trainer.evaluate(encoded_dataset["test"])

In [ ]:
out='/content/drive/MyDrive/Final_xlmroberta_casualmodel'
output_model_file = os.path.join(out, 'pytorch_model.bin')
output_config_file = os.path.join(out, 'config.json')
model_to_save = loaded_model.module if hasattr(loaded_model, 'module') else loaded_model
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(out) 